In [1]:
import cvxpy as cp
import numpy as np
import cmath
import hsbalance as hs
import matplotlib.pyplot as plt
%matplotlib widget

Using the Community license in this session. If you have a full Xpress license, first set the XPAUTH_PATH environment variable to the full path to your license file, xpauth.xpr, and then restart Python. If you want to use the FICO Community license and no longer want to see this message, set the XPAUTH_PATH environment variable to: /home/maged/anaconda3/envs/balance/lib/python3.8/site-packages/xpress/license/community-xpauth.xpr
NB: setting XPAUTH_PATH will also affect any other Xpress products installed on your system.


# Introduction

In these notes, we are going to discuss the issues related to robust optimization.
a. What is Robust Optimization?
Optimization process is not determinant, this means that we have no clue or determinant check if our solution is the optimum one. Optimization is prone to error if outliers exist. Noisy data and uncertainty readings. Methods, especially least squares method, can be more effected with outliers than others.  
b. Rotor Balancing Robust Optimization:
In our problem of rotor balancing, solution can be greatly effected by outliers for the following reasons:  
1. Measurements are low in numbers: Maximum field measuring points number is still tiny mathematically-wise.
2. Sensor faults: a sensor can go wrong easily for a turbine that runs for 20 years. Calibration is not an easy or a cheap process.  
3. Accumulation of errors: The process of measuring a point in a turbine/compressor usually involves a sensor and long cables in series (vibration cards and junctions). Those produce errors in series and cause the summation to infiltrate.
4. Measuring times: Turbo-Machinery is often connected to large process, which make it start and stop time costly. This encourages for accurate balancing with the fewest number of starts.

# Simulating Data

We generate random data with known solution W

In [32]:
np.random.seed(42)
real = np.random.rand(5,3)
imag = np.random.rand(5,3)
alpha = real + imag * 1j
# Assume W is equal to 10 grams @ 0 degree at all planes
W = [[10], [10], [10]]
# Create intial condtioning column array A with random noise to simulate actual problem
A = - alpha @ W + np.random.normal(1, size=(5,1))

# Create a hsbalance model

In [33]:
model_alpha = hs.Alpha()
model_alpha.add(alpha)
model = hs.LeastSquares(A, model_alpha)
W_model = model.solve()
hs.convert_cart_math(W_model)

array([['9.124@2.2'],
       ['10.11@358.7'],
       ['10.072@1.9']], dtype='<U11')

Calculate the Error

In [34]:
hs.rmse(W_model - W)

0.5115

# Faulty Sensor

To simulate a faulty sensor we will assume that the sensor at the first plane has gone completely fault and gives no output (i.e. equal 0)

In [35]:
A[0] -= A[0] * 0.2
model_alpha = hs.Alpha()
model_alpha.add(alpha)
model = hs.LeastSquares(A, model_alpha)
W_model = model.solve()
hs.convert_cart_math(W_model)

array([['9.533@5.9'],
       ['8.715@351.2'],
       ['9.312@7.2']], dtype='<U11')

In [36]:
hs.rmse(W_model - W)

1.4745

As we see, the error has gone extremly high and obviously unacceptable.  
If we acknowldge the sensor error, we can use weight least squares method to solve the problem instead.  
This can be done by assign the argument solver='WLS' in solver method of least squares. and pass the coefficient column array C to the model.   
In our case we will give all the sensor the same weight as 1. Only the sensor suspected we will assume a weight of 0. The previous setup will completely discard the sensor from caluclations.

In [37]:
C = np.ones(A.shape)
C[0] = 0
model = hs.LeastSquares(A, model_alpha, C)
W_model = model.solve(solver='WLS')
hs.convert_cart_math(W_model)

array([['9.272@2.6'],
       ['9.699@358.5'],
       ['10.032@3.4']], dtype='<U11')

In [38]:
hs.rmse(W_model - W)

0.6087

We can notice that we restored a low level of error when discarding the faulty sensor.

In [39]:
model = hs.LeastSquares(A, model_alpha)
W_model = model.solve(solver='huber')
hs.convert_cart_math(W_model)

array([['8.775@5.2'],
       ['9.082@349.6'],
       ['9.38@10.6']], dtype='<U11')

In [40]:
hs.rmse(W_model - W)

1.7766